In [49]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [50]:
def bilinear(img, ax=1., ay=1.):
	if len(img.shape) == 2:
		H,W = img.shape
		dim = 2
	else:
		H, W, C = img.shape
		dim = 3

	aH = int(ay * H)
	aW = int(ax * W)

	# position of resized image
	y = np.arange(aH).repeat(aW).reshape(aH, -1)
	x = np.tile(np.arange(aW), (aH, 1))

	# position of original position
	y = (y / ay)
	x = (x / ax)

	ix = np.floor(x).astype(np.int)
	iy = np.floor(y).astype(np.int)

	ix = np.minimum(ix, W-2)
	iy = np.minimum(iy, H-2)

	# distance 
	dx = x - ix
	dy = y - iy

	dx = np.repeat(np.expand_dims(dx, axis=-1), 3, axis=-1)
	dy = np.repeat(np.expand_dims(dy, axis=-1), 3, axis=-1)
	
	def weight(x,y):
		return x*y

	# interpolate
	out = weight((1-dx),(1-dy)) * img[iy, ix] + weight(dx,(1 - dy)) * img[iy, ix+1] + weight((1 - dx),dy) * img[iy+1, ix] + weight(dx,dy) * img[iy+1, ix+1]
	out = np.clip(out, 0, 255)
	out = out.astype(np.uint8)

	return out



In [51]:
def bicubic_mod(img, ax=1., ay=1.):
	if len(img.shape) == 2:
		H,W = img.shape
		dim = 2
	else:
		H, W, C = img.shape
		dim = 3

	aH = int(ay * H)
	aW = int(ax * W)

	# position of resized image
	y = np.arange(aH).repeat(aW).reshape(aH, -1)
	x = np.tile(np.arange(aW), (aH, 1))

	# position of original position
	y = (y / ay)
	x = (x / ax)

	ix = np.floor(x).astype(np.int)
	iy = np.floor(y).astype(np.int)

	ix = np.minimum(ix, W-2)
	iy = np.minimum(iy, H-2)

	# distance 
	dx = x - ix
	dy = y - iy

	dx = np.repeat(np.expand_dims(dx, axis=-1), dim, axis=-1)
	dy = np.repeat(np.expand_dims(dy, axis=-1), dim, axis=-1)
	
	def weight(x,y):
		return x**2 * x**2 * (9 - 6*x - 6*y + 4*x*y)

	# interpolate
	out = weight((1-dx),(1-dy)) * img[iy, ix] + weight(dx,(1 - dy)) * img[iy, ix+1] + weight((1 - dx),dy) * img[iy+1, ix] + weight(dx,dy) * img[iy+1, ix+1]
	out = np.clip(out, 0, 255)
	out = out.astype(np.uint8)

	return out



In [52]:
def bicubic(img, ax=1., ay=1.):
	if len(img.shape) == 2:
		H,W = img.shape
		dim = 2
	else:
		H, W, C = img.shape
		dim = 3

	aH = int(ay * H)
	aW = int(ax * W)

	# positions of resized image
	y = np.arange(aH).repeat(aW).reshape(aH, -1)
	x = np.tile(np.arange(aW), (aH, 1))
	y = (y / ay)
	x = (x / ax)

	# positions of original image
	ix = np.floor(x).astype(np.int)
	iy = np.floor(y).astype(np.int)

	ix = np.minimum(ix, W-1)
	iy = np.minimum(iy, H-1)

	# distance of each position of original image
	dx2 = x - ix
	dy2 = y - iy
	dx1 = dx2 + 1
	dy1 = dy2 + 1
	dx3 = 1 - dx2
	dy3 = 1 - dy2
	dx4 = 1 + dx3
	dy4 = 1 + dy3

	dxs = [dx1, dx2, dx3, dx4]
	dys = [dy1, dy2, dy3, dy4]

	def weight(t):
		a = -1.
		at = np.abs(t)
		w = np.zeros_like(t)
		ind = np.where(at <= 1)
		w[ind] = ((a+2) * np.power(at, 3) - (a+3) * np.power(at, 2) + 1)[ind]
		ind = np.where((at > 1) & (at <= 2))
		w[ind] = (a*np.power(at, 3) - 5*a*np.power(at, 2) + 8*a*at - 4*a)[ind]
		return w

	w_sum = np.zeros((aH, aW, C), dtype=np.float32)
	out = np.zeros((aH, aW, C), dtype=np.float32)

	# interpolate
	for j in range(-1, 3):
		for i in range(-1, 3):
			ind_x = np.minimum(np.maximum(ix + i, 0), W-1)
			ind_y = np.minimum(np.maximum(iy + j, 0), H-1)

			wx = weight(dxs[i+1])
			wy = weight(dys[j+1])
			wx = np.repeat(np.expand_dims(wx, axis=-1), 3, axis=-1)
			wy = np.repeat(np.expand_dims(wy, axis=-1), 3, axis=-1)

			w_sum += wx * wy
			out += wx * wy * img[ind_y, ind_x]

	out /= w_sum
	out = np.clip(out, 0, 255)
	out = out.astype(np.uint8)

	return out

In [55]:
# Read image
img = np.asarray(Image.open("original.jpeg"), dtype=np.float)
print(img.shape)
# Bilinear interpolation
ratio = 1.2
out = bilinear(img, ax=ratio, ay=ratio)
out2 = bicubic_mod(img, ax=ratio, ay=ratio)
out3 = bicubic(img, ax=ratio, ay=ratio)
Image.fromarray(out).save("bilinear.png")
Image.fromarray(out).save("bicubic_mod.png")
Image.fromarray(out).save("bicubic.png")

(128, 128, 3)
